In [5]:
import os
import cv2
import mediapipe as mp

# Define function to process image
def process_img(img, face_detection):
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)
    if out.detections is not None:
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box
            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height
            x1 = int(x1 * W)
            y1 = int(y1 * H)
            w = int(w * W)
            h = int(h * H)
            img[y1:y1 + h, x1:x1 + w, :] = cv2.blur(img[y1:y1 + h, x1:x1 + w, :], (50, 50))
    return img

# Define input arguments
args = {
    'mode': 'image',
    'filePath': 'womenFace.jpg'
}
# args = {
#     'mode': 'video',
#     'filePath': 'video.mp4'
# }
# args = {
#     'mode': 'webcam',
#     'filePath': None
# }

# Create output directory
output_dir = 'E:\computer vision'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Detect faces
mp_face_detection = mp.solutions.face_detection
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:

    if args['mode'] == "image":
        # Read image
        img = cv2.imread(args['filePath'])
        img = process_img(img, face_detection)
        # Save image
        cv2.imwrite(os.path.join(output_dir, 'output.png'), img)

    elif args['mode'] == 'video':
        # Read video file
        cap = cv2.VideoCapture(args['filePath'])
        ret, frame = cap.read()
        # Set up video writer
        output_video = cv2.VideoWriter(os.path.join(output_dir, 'output.mp4'),
                                       cv2.VideoWriter_fourcc(*'MP4V'),
                                       25,
                                       (frame.shape[1], frame.shape[0]))
        while ret:
            frame = process_img(frame, face_detection)
            output_video.write(frame)
            ret, frame = cap.read()
        cap.release()
        output_video.release()

    elif args['mode'] == 'webcam':
        # Capture from webcam
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        while ret:
            frame = process_img(frame, face_detection)
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            ret, frame = cap.read()
        cap.release()
        cv2.destroyAllWindows()